In [1]:
from tqdm.notebook import tqdm
from time import sleep
import numpy as np
import pandas as pd
import requests
import json

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from src.reviews import *

In [2]:
project = 'sahara'

In [3]:
df_commits = pd.read_csv(f'../data/commits/commits_{project}.csv')

In [4]:
len(df_commits)

3366

Change IDs

In [5]:
get_prefix = f"https://review.opendev.org/projects/openstack%2F{project}/commits/"

In [6]:
df_commits['change_id'] = ''

In [7]:
with tqdm(total = len(df_commits)) as pbar:
    for i, row in df_commits.iterrows():
        pbar.set_postfix({'hash' :  row['hash']})
        get_addr = get_prefix + row['hash']
        response = requests.get(get_addr)
        response_json = json.loads(response.text[5:]) # some leading nonsense
        message = response_json['message']
        change_id = get_change_id(message)
        if change_id == '':
            print("---hash {} does not contain a change id in message:\n{}".format(
                row['hash'], message))
        df_commits.at[i, 'change_id'] = change_id
        # Progress bar
        pbar.update(1)
        sleep(0.001)

  0%|          | 0/3366 [00:00<?, ?it/s]

---hash 9e78076745f2a7a23007e1d0c572eb1b8c20b12f does not contain a change id in message:
Initial commit

---hash 553449ac80a90c397c1f3a751ba6917167f38361 does not contain a change id in message:
install_venv fixed

---hash 90115d2b197b778d6d85efd77db3a541b8fa8eac does not contain a change id in message:
Initial implementation of REST API

---hash 594d6919bacb68f256993af6d27106fecdc16f80 does not contain a change id in message:
Some fixes

---hash fc3ea9c5eace75cc3498f940b20ffe946335bfec does not contain a change id in message:
bin added

---hash 4f51cdaa56294d277661049a23cf24bed35dde38 does not contain a change id in message:
incorrect scheduler call has been removed

---hash 79fbe627b3e3215ceb5e51fabbd10ef27027df1b does not contain a change id in message:
note about hooks added

---hash 51e6e2b04fd8f9159352f4b736e08b1b5f691b76 does not contain a change id in message:
tests, coverage added

---hash c1903c767dc8dbbda7da7169d6658fd6619c6d67 does not contain a change id in message:
Readm

---hash 7948196741dd8871d53726b659fabb175db38781 does not contain a change id in message:
some fixes, clusterops are now starting using eventlet

---hash 7e9c8aac89f0fb9e6849ec580979967525e2e093 does not contain a change id in message:
todo added

---hash 37b43ac2f32eae55e4a760619a5ad58a39239c54 does not contain a change id in message:
logging added

---hash aa5629cccfb70419356e9f4f3e2bcaa68e5bb2c3 does not contain a change id in message:
api test has been upgraded

---hash 370a3ac563481861dd762ecefc87e8bb7159c906 does not contain a change id in message:
nodes are now sorted before assertEquals while creating clusters

---hash 4dbd12582e26bc4a79275806be2306751aa9792e does not contain a change id in message:
warnings has been fixed

---hash 22a101f286a9bacf2f196e5296d4db4e35b6c3cc does not contain a change id in message:
allow cluster ops flag added

---hash 0ad7be0f7ed0f7cb211da74ff39074c4c27d7aa4 does not contain a change id in message:
tests has been fixed

---hash f5c01df1af5ab1397b

---hash 39f08ef46d5ef46d6c4f4c2999b458eb5d3e4833 does not contain a change id in message:
if content type is undefined json should be applied

---hash 8747d48fc01bdaed34ff8f00ca4da617b99b52e3 does not contain a change id in message:
Corrected API docs

---hash ab43ce80b16db0c975352e301ae137e032cbc28f does not contain a change id in message:
Merge pull request #5 from Mirantis/docs_corrections

Corrected API docs
---hash b9ef3e05dd05d7a7c16d80dc17ddc4f3b78592ee does not contain a change id in message:
default node_password has been changed

---hash 0edfcc107a6278b74ce8fb8114ce690498cf59fd does not contain a change id in message:
roadmap has been updated

---hash 6a735f1780260373273d13717b7f443a0d6a5d38 does not contain a change id in message:
custom horizon screenshots has been added to docs

---hash 356acac106942c1bc68ecb48eb2d8a6641b942bf does not contain a change id in message:
Corrected examples in API docs

---hash c365288219b50f6a142b4067a7a1b7612c241b6c does not contain a change 

---hash e23d985387901054b65f49cc79817809d83d39bc does not contain a change id in message:
eho -> savanna

---hash 679f1400b9b631ade0287b404b2d321b756d19c9 does not contain a change id in message:
Changed docs with replacement of EHO to Savanna

---hash 0372a12341d253a66175c87404ff7e448c028dab does not contain a change id in message:
Changed pictures in docs according to Savanna name and replaced Horizon pages

---hash 2ab39e152027d98fb5101f024ef37c36d76220d0 does not contain a change id in message:
mailing list address has been fixed

---hash 4543a4c5a2280ca51517ff4e8e9a3929f1cbd1bf does not contain a change id in message:
Add .gitreview file.

---hash e09194638bd2b26f6c5cada9342de216385a44db does not contain a change id in message:
conf files moved to the right place

---hash 88f82f97b2ab7c0799f66f0da091c147c2f75eaf does not contain a change id in message:
renaming rollbacked to prevent problems with the old image

---hash 56a7a33348b29815f7d01c7db993b0f2c6e3d74e does not contain a ch

In [8]:
np.sum(df_commits['change_id'] == '')

199

In [9]:
df_commits = df_commits.loc[df_commits['change_id'] != '']

Change Messages

In [10]:
get_prefix = f'https://review.opendev.org/changes/openstack%2F{project}~master~'

In [11]:
reviews = []

In [12]:
with tqdm(total = len(df_commits)) as pbar:
    for i, row in df_commits.iterrows():
        pbar.set_postfix({'hash' :  row['hash']})
        get_addr = get_prefix + row['change_id'] + "/messages"
        response = requests.get(get_addr)
        if response.status_code == 404:
            print("---hash {} with change id {} had invalid response".format(
                row['hash'], row['change_id']))
        else:
            response_json = json.loads(response.text[5:])
            reviews_for_single_change = get_review_info(project, response_json, row['author_name'])
            for re in reviews_for_single_change:
                re['hash'] = row['hash']
                re['change_id'] = row['change_id']
            reviews.extend(reviews_for_single_change)
        # Progress bar
        pbar.update(1)
        sleep(0.001)

  0%|          | 0/3167 [00:00<?, ?it/s]

---hash 7a6e99948dd524dc86561a8810e9ed517c8626c1 with change id Ia09f96ca27d4f6683c5e62f0040f9e68434680b7 had invalid response
---hash 0cc104aa37a157b987895d17b018b34600701c8b with change id Ia187bb71df998908c40281f71003d7cf3d092dd2 had invalid response
---hash 6daf4c0b662dd151360ef1afaef4040b20b4d633 with change id I4b22a865e3885dcf529e685c05ab64b3855699da had invalid response


Save to output

In [13]:
df_reviews = pd.DataFrame(reviews)

In [14]:
df_reviews.to_csv(f"../data/reviews/reviews_{project}.csv", index = False)